In [0]:
from pyspark.sql.functions import *
import urllib


class S3Connector:
    def __init__(self, config_dict):
        self._s3_bucket_name = config_dict["s3_bucket_name"]
        self._bucket_mount_name = config_dict["target_bucket_mount_name"]
        self._aws_credentials_filepath = config_dict["aws_authentication_csv_filepath"]
        self._is_bucket_mounted = self.check_if_bucket_already_mounted()

    def check_if_bucket_already_mounted(self):
        list_mounts = dbutils.fs.mounts()
        mount_points = map(lambda x: x[0], list_mounts)
        return self._bucket_mount_name in list(mount_points)

    def __read_AWS_authentication_details_to_df(self):
        file_type = "csv"
        first_row_is_header = "true"
        delimiter = ","
        aws_keys_df = spark.read.format(file_type)\
                                .option("header", first_row_is_header)\
                                .option("sep", delimiter)\
                                .load(self._aws_credentials_filepath)
                                
        return aws_keys_df

    def _extract_access_credentials(self):
        
        aws_keys_df = self.__read_AWS_authentication_details_to_df()

        # Get the AWS access key and secret key from the spark dataframe
        ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
        
        SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
        
        # Encode the secret key
        ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")
        
        return (ACCESS_KEY, ENCODED_SECRET_KEY)

    def mount_S3_bucket(self):

        if self._is_bucket_mounted:
            print(f"Your S3 bucket has already been mounted under the name of {self._bucket_mount_name}")
        else:
            aws_credentials_tuple = self._extract_access_credentials()

            ACCESS_KEY = aws_credentials_tuple[0]
            ENCODED_SECRET_KEY = aws_credentials_tuple[1]

            SOURCE_URL = f"s3n://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{self._s3_bucket_name}"

            dbutils.fs.mount(SOURCE_URL, self._bucket_mount_name)
            self._is_bucket_mounted = True
    
    def unmount_S3_bucket(self):
        dbutils.fs.unmount(self._bucket_mount_name)
        self._is_bucket_mounted = False